In [1]:
pwd

'/mmfs1/data/aglinska/BC-fMRI-AE/Notebooks'

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='3'

In [3]:
import numpy as np
import pandas as pd 
import pickle
from helper_funcs import *
from umap import UMAP

from matplotlib import pyplot as plt
import seaborn as sns

In [4]:
data_dir = '../Data/latent_vecs/'
files = [file for file in os.listdir(data_dir) if file.startswith('latent')]
files.sort()
files

['latent_vecs_CVAE_2021-09-15 14:03:53.826090',
 'latent_vecs_CVAE_2021-09-30 17:08:46.471890']

In [14]:
weights = os.listdir('../Assets/tf_weights/')
weights.sort()
weights

['.ipynb_checkpoints',
 'CVAE_2021-09-15 07:07:52.059178',
 'CVAE_2021-09-15 07:16:36.484543',
 'CVAE_2021-09-15 07:19:23.138115',
 'CVAE_2021-09-15 14:03:53.826090',
 'CVAE_2021-09-16 16:34:00.330135',
 'CVAE_2021-09-30 15:00:00.089858',
 'CVAE_2021-09-30 15:41:40.385003',
 'CVAE_2021-09-30 17:08:46.471890',
 'CVAE_2021-10-01 08:19:19.567960',
 'CVAE_2021-10-05 16:12:30.839342',
 'CVAE_2021-10-06 08:06:32.222665',
 'CVAE_2021-10-06 08:08:11.612788',
 'CVAE_2021-10-06 08:35:55.310173',
 'CVAE_2021-10-06 09:18:26.408246',
 'CVAE_2021-10-06 09:19:00.166747']

In [15]:
weights_folder = 'CVAE_2021-10-06 08:35:55.310173'

In [16]:
from importlib import reload
import make_models;reload(make_models);from make_models import *

batch_size = 32
cvae, z_encoder, s_encoder, cvae_decoder = get_fMRI_CVAE_4D(input_shape=(32,32,32,51),
                                                             latent_dim=16,
                                                             beta=1,
                                                             gamma=1,
                                                             disentangle=True,
                                                             bias=True,
                                                             batch_size = batch_size,
                                                             kernel_size = 3,
                                                             filters = 8,
                                                             intermediate_dim = 128,
                                                             nlayers = 5,
                                                             learning_rate=0.001,
                                                             opt=None)

In [17]:
cvae.load_weights(os.path.join('../Assets/tf_weights/',weights_folder,'cvae_weights'));

In [20]:
df = pd.read_csv('../Data/ABIDE_df2.csv')
df.head()

,participant_id,DX_GROUP,DSM_IV_TR,AGE_AT_SCAN,SEX,HANDEDNESS_CATEGORY,HANDEDNESS_SCORES,FIQ,VIQ,PIQ,...,WISC_IV_MATRIX_SCALED,WISC_IV_DIGIT_SPAN_SCALED,WISC_IV_LET_NUM_SCALED,WISC_IV_CODING_SCALED,WISC_IV_SYM_SCALED,EYE_STATUS_AT_SCAN,AGE_AT_MPRAGE,BMI,bids_folder,site
0,50002,1,1.0,16.77,1,Ambi,NaN,103.0,116.0,89.0,...,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,sub-Pitt0050002,13
1,50003,1,1.0,24.45,1,R,NaN,124.0,128.0,115.0,...,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,sub-Pitt0050003,13
2,50004,1,1.0,19.09,1,R,NaN,113.0,108.0,117.0,...,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,sub-Pitt0050004,13
3,50005,1,1.0,13.73,2,R,NaN,119.0,117.0,118.0,...,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,sub-Pitt0050005,13
4,50006,1,1.0,13.37,1,L,NaN,109.0,99.0,119.0,...,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,sub-Pitt0050006,13


In [27]:
data_dir = '../Assets/fc_mats_32smooth_new'
s = 0
sub = df['bids_folder'].values[s]
datum = np.load(os.path.join(data_dir,sub+'.npy'))/2

In [29]:
from tqdm import tqdm
data = np.array([np.load(os.path.join(data_dir,sub+'.npy'))/2 for sub in tqdm(df['bids_folder'].values)])

100%|██████████| 1048/1048 [00:11<00:00, 91.36it/s]


In [30]:
data.shape

(1048, 32, 32, 32, 51)

In [34]:
data_batch = data[0:32,:,:,:,:]

In [35]:
asd_out, td_out = cvae.predict([data_batch,data_batch])

[array([[[[[0.49962276, 0.49962384, 0.49971318, ..., 0.49986413,
            0.49990907, 0.5000089 ],
           [0.49960157, 0.49975824, 0.49985495, ..., 0.49979582,
            0.49987975, 0.5000345 ],
           [0.499469  , 0.4997123 , 0.4998609 , ..., 0.4997762 ,
            0.4998813 , 0.49995327],
           ...,
           [0.499543  , 0.49971217, 0.4998415 , ..., 0.49979672,
            0.49989954, 0.50003153],
           [0.49947014, 0.49972972, 0.4998819 , ..., 0.49977943,
            0.49988797, 0.49999595],
           [0.49955627, 0.49974352, 0.49984092, ..., 0.4998065 ,
            0.49996573, 0.50010043]],
 
          [[0.49986604, 0.49990717, 0.4999129 , ..., 0.49975383,
            0.4998284 , 0.49994493],
           [0.4997352 , 0.5000868 , 0.5001348 , ..., 0.49969292,
            0.49974394, 0.4997455 ],
           [0.49979088, 0.5002618 , 0.50029147, ..., 0.4995335 ,
            0.49968892, 0.49981058],
           ...,
           [0.4996649 , 0.50002944, 0.5001361 ,